In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [2]:
def process_csv_file(file):
    csv_data = pd.read_csv(file)
    csv_data = csv_data.drop("class", axis = 1)
    return (csv_data.to_numpy())

In [3]:
def set_padding(arr, max_length):
    arr = np.append(arr, np.zeros((max_length-arr.shape[0],300)), axis=0)
    return np.expand_dims(arr, 0)

In [4]:
def prepare_data(folder_path, labels):
    total = 0
    max_length = 0
    # find max length
    for folder in os.listdir(folder_path):
        if folder != ".DS_Store":
            total += len(os.listdir(os.path.join(folder_path, folder)))
            for file in os.listdir(os.path.join(folder_path, folder)):
                data = process_csv_file(os.path.join(folder_path, folder, file))
                if data.shape[0] > max_length:
                    max_length = data.shape[0]
    # Make the arrays
    X = np.empty((0, max_length, 300))
    Y = np.empty((0,), int)
    for folder in os.listdir(folder_path):
        print(folder)
        if folder != ".DS_Store":
            for file in os.listdir(os.path.join(folder_path, folder)):
                data = process_csv_file(os.path.join(folder_path, folder, file))
                data = set_padding(data, max_length)
                X = np.vstack((X, data))
                Y = np.append(Y, labels[folder])
    print(total, max_length)
    return X, Y


In [8]:
X[0]

array([[ 0.49718088,  0.25081992, -0.48320499, ...,  0.77923   ,
        -0.05017287,  0.        ],
       [ 0.49774376,  0.25079364, -0.77710956, ...,  0.79552472,
        -0.05187845,  0.        ],
       [ 0.49550352,  0.25237963, -0.72304803, ...,  0.80509031,
        -0.04531861,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [11]:
folder_path = "/Users/aly/Documents/Programming/Apps/Machine Learning/ASL Converter/training_models/mediapipe/reformatting-the-data/data_four_labels/"
labels= {"coffee": 0, 'dog': 1, 'milk': 2, 'door': 3}
X, Y = prepare_data(folder_path, labels)

.DS_Store
dog
milk
coffee
door
143 211


In [6]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
], padding="post"
)
# print(X[0][1])
print(((padded_inputs)))

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


In [21]:
# Y = np.expand_dims(Y, axis=0)
# Y= Y.T
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [12]:
# split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [13]:
X_train.shape, y_train.shape

((114, 211, 300), (114,))

In [15]:
x_shape = X_train.shape

In [25]:
from tensorflow.keras import layers
masking_input = []
for i in range(x_shape[0]):
    masking_input.append(X_train[i].T[0])

masking_input = np.array(masking_input)

masking_input.shape

(114, 211)

In [27]:
embedding = layers.Embedding(input_dim=x_shape[1], output_dim=x_shape[1], mask_zero=True)
masked_output = embedding(masking_input)

print(masked_output._keras_mask)
masked_output
# masking_layer = layers.Masking()
# # Simulate the embedding lookup by expanding the 2D input to 3D,
# # with embedding dimension of 10.
# unmasked_embedding = tf.cast(
#     tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
# )

# masked_embedding = masking_layer(unmasked_embedding)
# print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(114, 211), dtype=bool)


<tf.Tensor: shape=(114, 211, 211), dtype=float32, numpy=
array([[[-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        ...,
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075]],

       [[-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        [-0.0188207 ,  0.03263413,  0.01849456, ...,  0.00914326,
         -0.02826176,  0.00322075],
        ..

In [30]:
#LSTM model 

inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.Masking(mask_value=0, input_shape=(x_shape[1], x_shape[2]))(inputs)
x = layers.LSTM(64)(x)
# x = layers.MaxPool2D()(x)
outputs = layers.Dense(4, activation="softmax")(x)

model_lstm = tf.keras.Model(inputs, outputs)
model_lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 211, 300)]        0         
                                                                 
 masking_2 (Masking)         (None, 211, 300)          0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                93440     
                                                                 
 dense (Dense)               (None, 4)                 260       
                                                                 
Total params: 93,700
Trainable params: 93,700
Non-trainable params: 0
_________________________________________________________________


In [31]:
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5


2023-03-25 15:30:05.484563: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-25 15:30:07.063605: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-25 15:30:07.492899: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_24'
2023-03-25 15:30:07.496838: I tensorflow/core/grappler/optimizers/cu

4/4 [==============================] - ETA: 0s - loss: 1.4054 - accuracy: 0.3070 

2023-03-25 15:31:36.191353: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-25 15:31:36.418843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 94s 16s/step - loss: 1.4054 - accuracy: 0.3070 - val_loss: 1.2376 - val_accuracy: 0.4138
Epoch 2/5
4/4 [==============================] - 129s 29s/step - loss: 1.3135 - accuracy: 0.3947 - val_loss: 1.1903 - val_accuracy: 0.3793
Epoch 3/5
4/4 [==============================] - 173s 43s/step - loss: 1.2444 - accuracy: 0.4123 - val_loss: 1.0347 - val_accuracy: 0.4483
Epoch 4/5
4/4 [==============================] - 91s 16s/step - loss: 1.2728 - accuracy: 0.3860 - val_loss: 0.9785 - val_accuracy: 0.4828
Epoch 5/5
4/4 [==============================] - 179s 45s/step - loss: 1.0759 - accuracy: 0.4825 - val_loss: 0.9250 - val_accuracy: 0.5517


In [47]:
def get_mask_from_lengths(data):  #2d list
    ids = tf.zeros(shape=(x_shape[1], x_shape[2]))
    all_zeros = ids < data
    mask_list = []
    for i in range()
    return mask

get_mask_from_lengths(X_train)[0]

<tf.Tensor: shape=(211, 300), dtype=bool, numpy=
array([[ True,  True, False, ...,  True, False, False],
       [ True,  True, False, ...,  True, False, False],
       [ True,  True, False, ...,  True, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [48]:
inputs = layers.Input(shape=(x_shape[1], x_shape[2]))
x = layers.GRU(64)(inputs, mask=masked_output._keras_mask)
# x = layers.LSTM(64)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model_lstm2= tf.keras.Model(inputs, outputs)
model_lstm2.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 211, 300)]        0         
                                                                 
 gru (GRU)                   (114, 64)                 70272     
                                                                 
 dense_3 (Dense)             (114, 4)                  260       
                                                                 
Total params: 70,532
Trainable params: 70,532
Non-trainable params: 0
_________________________________________________________________


In [49]:
model_lstm2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm2.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5


2023-03-25 16:08:52.820913: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-25 16:08:53.187428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node model_3/dense_3/Softmax/_20}}]] [Op:__inference_train_function_36100]

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])




Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-20 13:48:08.831618: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-20 13:48:08.831726: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'X_train' is not defined

In [41]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_33 (LSTM)              (None, 211, 64)           93440     
                                                                 
 lstm_34 (LSTM)              (None, 32)                12416     
                                                                 
 dense_7 (Dense)             (None, 4)                 132       
                                                                 
Total params: 105,988
Trainable params: 105,988
Non-trainable params: 0
_________________________________________________________________


In [42]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/10


2023-03-18 02:36:51.372559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:51.660659: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:51.934093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:52.419899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:53.015099: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - ETA: 0s - loss: 1.3853 - accuracy: 0.2353

2023-03-18 02:36:55.555320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:55.667921: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-18 02:36:55.926320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 6s 863ms/step - loss: 1.3853 - accuracy: 0.2353 - val_loss: 1.3813 - val_accuracy: 0.3333
Epoch 2/10
4/4 [==============================] - 0s 108ms/step - loss: 1.3759 - accuracy: 0.3333 - val_loss: 1.3774 - val_accuracy: 0.3333
Epoch 3/10
4/4 [==============================] - 0s 99ms/step - loss: 1.3661 - accuracy: 0.3431 - val_loss: 1.3739 - val_accuracy: 0.3333
Epoch 4/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3470 - accuracy: 0.3431 - val_loss: 1.3667 - val_accuracy: 0.3333
Epoch 5/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3325 - accuracy: 0.3431 - val_loss: 1.3640 - val_accuracy: 0.3333
Epoch 6/10
4/4 [==============================] - 0s 101ms/step - loss: 1.3215 - accuracy: 0.3431 - val_loss: 1.4380 - val_accuracy: 0.3333
Epoch 7/10
4/4 [==============================] - 0s 102ms/step - loss: 1.2628 - accuracy: 0.4118 - val_loss: 2.1948 - val_accuracy: 0.1667
Epoch 8/10
4/4 [================

In [38]:
x = np.array([[4], [2]])

In [41]:
x.T.shape

(1, 2)

In [42]:
x.T

array([[4, 2]])